<a href="https://colab.research.google.com/github/BitLorax/depth-prediction/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import os
import random

In [0]:
!pip install -q git+https://github.com/tensorflow/examples.git
from tensorflow_examples.models.pix2pix import pix2pix

# SSH

In [0]:
!rm -rf /root/.ssh
!mkdir /root/.ssh

In [0]:
!tar xvzf ssh.tar.gz

In [0]:
!cp ssh-colab/* /root/.ssh && rm -rf ssh-colab && rm -rf ssh.tar.gz
!chmod 700 /root/.ssh

In [0]:
!touch /root/.ssh/known_hosts
!ssh-keyscan github.com >> /root/.ssh/known_hosts
!chmod 644 /root/.ssh/known_hosts

# Download Data

In [0]:
!git config --global user.email 'willjhliang@gmail.com'
!git config --global user.name 'BitLorax'

In [0]:
!rm -r .git

In [0]:
!git init
!git remote add -f origin git@github.com:BitLorax/depth-prediction.git

!git config core.sparseCheckout true
!echo 'nyuDepth' >> .git/info/sparse-checkout

In [0]:
!git pull origin master

# Data Generator

In [0]:
M = 1449
TRAIN_SPLIT = .9
m = (int)(M * TRAIN_SPLIT)

BATCH_SIZE = 16

width, height = 128, 128

In [0]:
class DataGenerator(tf.keras.utils.Sequence):
  def __init__(self, listIDs, batchSize, dim, nChannels):
    self.list_IDs = listIDs
    self.batch_size = batchSize
    self.dim = dim
    self.n_channels = nChannels


  def on_epoch_end(self):
    np.random.shuffle(self.list_IDs)

  
  def __len__(self):
    return int(np.floor(len(self.list_IDs) / self.batch_size))


  def __getitem__(self, idx):
    curIDs = self.list_IDs[idx*self.batch_size:(idx + 1)*self.batch_size]

    X = np.empty((self.batch_size, *self.dim, self.n_channels))
    y = np.empty((self.batch_size, *self.dim))

    for i, ID in enumerate(curIDs):
      img = np.load('nyuDepth/img' + str(ID) + '.npy')
      dep = np.load('nyuDepth/dep' + str(ID) + '.npy')

      # take out white border
      border = 4
      img = img[border:img.shape[0] - border, border:img.shape[1] - border]
      dep = dep[border:dep.shape[0] - border, border:dep.shape[1] - border]

      # get random subsection of size 128 x 128
      kx = random.randrange(0, img.shape[0] - width)
      ky = random.randrange(0, img.shape[1] - height)
      X[i,] = img[kx:kx + width][ky:ky + height]
      y[i,] = dep[kx:kx + width][ky:ky + height]

      # flip image horizontally
      if random.random() > .5:
        X[i,] = np.flip(X[i,], 1)
        y[i,] = np.flip(y[i,], 1)

    return X, y

In [0]:
trainGen = DataGenerator(np.arange(0, m), BATCH_SIZE, (128, 128), 3)
valGen = DataGenerator(np.arange(m, M), BATCH_SIZE, (128, 128), 3)

## Load Data (Backup)
Use in case Data Generator breaks

In [0]:
temp = np.load('nyuDepth/data0.npz')['images']
width = temp.shape[1]
height = temp.shape[2]

images = np.empty((0, width, height, 3))
depths = np.empty((0, width, height))

In [0]:
LOAD_START = 0
LOAD_END = 108

for i in range(LOAD_START, LOAD_END, 1):
  data = np.load('nyuDepth/data' + str(i) + '.npz')
  curImgs = data['images']
  curDeps = data['depths']
  images = np.concatenate((images, curImgs), axis=0)
  depths = np.concatenate((depths, curDeps), axis=0)

In [0]:
m = images.shape[0]

TRAINING_SPLIT = .9
idx = (int)(TRAINING_SPLIT * m)
imgTrain = images[:idx]
depTrain = depths[:idx]
imgTest = images[idx:]
depTest = depths[idx:]

In [0]:
def shuffle(images, depths):
  random = np.arange(images.shape[0])
  np.random.shuffle(random)
  images = images[random]
  depths = depths[random]
  return images, depths

In [0]:
imgTrain, depTrain = shuffle(imgTrain, depTrain)
imgTest, depTest = shuffle(imgTest, depTest)

In [0]:
print(images.shape)
print(depths.shape)
print()
print(imgTrain.shape)
print(depTrain.shape)
print()
print(imgTest.shape)
print(depTest.shape)

In [0]:
train = tf.data.Dataset.from_tensor_slices((imgTrain, depTrain)).repeat()
train = train.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test = tf.data.Dataset.from_tensor_slices((imgTest, depTest))

In [0]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test = test.batch(BATCH_SIZE)

# Build Model

In [0]:
vgg16 = tf.keras.applications.VGG16(input_shape=(width, height, 3),
                                    include_top=False,
                                    weights='imagenet')

xIn = tf.keras.Input(shape=(width, height, 3))

# block 1    128 x 128 x 3 -> 64 x 64 x 64
x = tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu', name='block1_conv1')(xIn)
out1 = tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu', name='block1_conv2')
x = out1(x)
x = tf.keras.layers.MaxPool2D(name='block1_pool')(x)
#x = tf.keras.layers.BatchNormalization(name='block1_bn')(x)

# block 2    64 x 64 x 64 -> 32 x 32 x 128
x = tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu', name='block2_conv1')(x)
out2 = tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu', name='block2_conv2')
x = out2(x)
x = tf.keras.layers.MaxPool2D(name='block2_pool')(x)
#x = tf.keras.layers.BatchNormalization(name='block2_bn')(x)

# block 3    32 x 32 x 128 -> 16 x 16 x 256
x = tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu', name='block3_conv1')(x)
x = tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu', name='block3_conv2')(x)
out3 = tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu', name='block3_conv3')
x = out3(x)
x = tf.keras.layers.MaxPool2D(name='block3_pool')(x)
#x = tf.keras.layers.BatchNormalization(name='block3_bn')(x)

# block 4    16 x 16 x 256 -> 8 x 8 x 512
x = tf.keras.layers.Conv2D(512, 3, padding='same', activation='relu', name='block4_conv1')(x)
x = tf.keras.layers.Conv2D(512, 3, padding='same', activation='relu', name='block4_conv2')(x)
out4 = tf.keras.layers.Conv2D(512, 3, padding='same', activation='relu', name='block4_conv3')
x = out4(x)
x = tf.keras.layers.MaxPool2D(name='block4_pool')(x)
#x = tf.keras.layers.BatchNormalization(name='block4_bn')(x)

# block 5    8 x 8 x 512 -> 4 x 4 x 512
x = tf.keras.layers.Conv2D(512, 3, padding='same', activation='relu', name='block5_conv1')(x)
x = tf.keras.layers.Conv2D(512, 3, padding='same', activation='relu', name='block5_conv2')(x)
out5 = tf.keras.layers.Conv2D(512, 3, padding='same', activation='relu', name='block5_conv3')
x = out5(x)
x = tf.keras.layers.MaxPool2D(name='block5_pool')(x)
#x = tf.keras.layers.BatchNormalization(name='block5_bn')(x)
downStack = tf.keras.Model(inputs=xIn, outputs=[out1.output, out2.output,
                                                out3.output, out4.output,
                                                out5.output, x])

# load weights from imagenet
layerNames = [layer.name for layer in downStack.layers]
vggLayerNames = [layer.name for layer in vgg16.layers]
vggLayerNames.pop(0)

for i, name in enumerate(vggLayerNames):
  downStack.layers[layerNames.index(name)].set_weights(vgg16.layers[i + 1].get_weights())
  print('Loaded weights for ' + name)

In [0]:
upFilterSizes = [512, 512, 256, 128, 64]
upSamples = [pix2pix.upsample(filterSize, 3) for filterSize in upFilterSizes]

In [0]:
def uNet():
  inputs = tf.keras.layers.Input(shape=(width, height, 3))
  x = inputs

  skips = downStack(x)
  x = skips[-1]
  skips = reversed(skips[:-1])

  for up, skip, filterSize in zip(upSamples, skips, upFilterSizes):
    x = up(x)
    x = tf.keras.layers.Concatenate()([x, skip])
    x = tf.keras.layers.Conv2D(filterSize, 3, padding='same', activation='relu')(x)
  
  last = tf.keras.layers.Conv2D(1, 3, padding='same')
  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

In [0]:
model = uNet()
model.compile(optimizer='adam',
              loss=tf.keras.losses.Huber())

In [0]:
tf.keras.utils.plot_model(model, show_shapes=True)

# Load Weights

In [0]:
model.load_weights('checkpoints/ckpt-43-0.375119.h5')

# Train Model

In [0]:
def display(dList):
  fig = plt.figure(figsize=(10, 10))
  for i in range(len(dList)):
    plt.subplot(1, len(dList), i + 1)
    plt.imshow(dList[i])
    plt.axis('off')
  plt.show()
  return fig

In [0]:
sampleImg = np.load('nyuDepth/img' + (str)(m) + '.npy')[border:border + width][border:border + height]
sampleDep = np.load('nyuDepth/dep' + (str)(m) + '.npy')[border:border + width][border:border + height]

In [0]:
def predict():
  return display([sampleImg.astype('uint8'), sampleDep,
                  model.predict(sampleImg[tf.newaxis, ...])[0, :, :, 0]])

In [0]:
predict()

In [0]:
class DisplayCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    clear_output(wait=True)
    predict()

In [0]:
if not os.path.exists('./checkpoints'):
  os.makedirs('./checkpoints')

In [0]:
checkpointCallback = tf.keras.callbacks.ModelCheckpoint('./checkpoints/ckpt.h5',
                                     monitor='val_loss',
                                     mode='min',
                                     verbose=1,
                                     save_freq='epoch',
                                     save_weights_only=True,
                                     save_best_only=True)

In [0]:
EPOCHS = 120
STEPS_PER_EPOCH = (m / BATCH_SIZE) - 1
VAL_STEPS = ((M - m) / BATCH_SIZE) - 1
hist = model.fit(trainGen,
                 epochs=EPOCHS,
                 steps_per_epoch = STEPS_PER_EPOCH,
                 validation_steps=VAL_STEPS,
                 validation_data=valGen,
                 callbacks=[DisplayCallback(), checkpointCallback])

In [0]:
pred = [sampleImg.astype('uint8'), sampleDep, model.predict(sampleImg[tf.newaxis, ...])[0, :, :, 0]]
fig = display(pred)
fig.savefig('result.png')

# Push Changes

In [0]:
!ls -t1 checkpoints | head -n 1

In [0]:
!git add checkpoints/ckpt-06-0.248222.h5
!git add result.png
!git commit -m "Checkpoint after 120 epochs"
!git push --set-upstream origin master